In [4]:
import sys
import os
import math
import euclid
import mesh
import geomgen
import fbx

In [5]:
sys.path = sys.path + [os.curdir + os.sep + "fbx2018_1"]    

In [6]:
 from FbxCommon import *

In [7]:
def create_sdk_manager():
    result = FbxManager.Create()
    return result

def create_scene(manager, name = "FbxScene"):
    ios = FbxIOSettings.Create(manager, IOSROOT)
    manager.SetIOSettings(ios)
    scene = FbxScene.Create(manager, name)
    return scene



In [8]:
def load_scene(manager, scene, file_name):
    importer = FbxImporter.Create(manager, "")    
    result = importer.Initialize(file_name, -1, manager.GetIOSettings())
    if not result:
        return False    
    if importer.IsFBX():
        manager.GetIOSettings().SetBoolProp(EXP_FBX_MATERIAL, True)
        manager.GetIOSettings().SetBoolProp(EXP_FBX_TEXTURE, True)
        manager.GetIOSettings().SetBoolProp(EXP_FBX_EMBEDDED, True)
        manager.GetIOSettings().SetBoolProp(EXP_FBX_SHAPE, True)
        manager.GetIOSettings().SetBoolProp(EXP_FBX_GOBO, True)
        manager.GetIOSettings().SetBoolProp(EXP_FBX_ANIMATION, True)
        manager.GetIOSettings().SetBoolProp(EXP_FBX_GLOBAL_SETTINGS, True)
    
    result = importer.Import(scene)
    importer.Destroy()
    return result

In [9]:
# Prepare the FBX SDK.
lSdkManager, lScene = InitializeSdkObjects()

In [10]:
lResult = LoadScene(lSdkManager, lScene, "Track.fbx")

In [11]:
 lRootNode = lScene.GetRootNode()

In [12]:
def get_node_geometric_transform(node):
    t = node.GetGeometricTransformation()

In [13]:
def print_node_info(node):
    print node.GetName()
    print "="
    
    print "="

In [14]:
def walk_nodes_aux(node):
    for i in range(node.GetChildCount()):
        print node.GetChild(i).GetName()
        walk_nodes_aux(node.GetChild(i))
        
def walk_nodes(scene):
    root = scene.GetRootNode()
    walk_nodes_aux(root)

In [15]:
walk_nodes(lScene)

Track_Node


In [16]:
lScene.Destroy()

In [17]:
scene = create_scene(lSdkManager, "Generic")

In [18]:
load_scene(lSdkManager, scene, "GenericMan.fbx" )

True

In [19]:
root_node = scene.GetRootNode()

In [20]:
walk_nodes(scene)

Character
Root
Waist
Left Hip
Left Knee
Left Foot
Right Hip
Right Knee
Right Foot
Middle Spine
Neck
Left Arm
Left Elbow
Left Hand
Head
Right Arm
Right Elbow
Right Hand


In [21]:
FbxTransform.__doc__

'FbxTransform()\nFbxTransform(FbxTransform)'

In [22]:
root_node.EvaluateGlobalTransform(FbxTime(0.0))

In [23]:
FbxTime(0.0)

In [24]:
gt = root_node.GetGeometricTranslation(FbxNode.eSourcePivot)
gr = root_node.GetGeometricRotation(FbxNode.eSourcePivot)
gs = root_node.GetGeometricScaling(FbxNode.eSourcePivot)

In [25]:
print gt, gr, gs

fbx.FbxVector4(0.000000, 0.000000, 0.000000, 1.000000) fbx.FbxVector4(0.000000, 0.000000, 0.000000, 1.000000) fbx.FbxVector4(1.000000, 1.000000, 1.000000, 1.000000)


In [26]:
FbxAMatrix(gt, gr, gs)

In [27]:
def get_geometry_transform(node):
    gt = node.GetGeometricTranslation(FbxNode.eSourcePivot)
    gr = node.GetGeometricRotation(FbxNode.eSourcePivot)
    gs = node.GetGeometricScaling(FbxNode.eSourcePivot)
    return FbxAMatrix(gt, gr, gs)

In [28]:
def get_global_transform(node, time = 0.0):
    t = FbxTime(time)
    return node.EvaluateGlobalTransform(t)

In [29]:
def get_local_transform(node, time = 0.0):
    t = FbxTime(time)
    return node.EvaluateLocalTransform(t)

In [30]:
print get_global_transform(root_node, 0.0)

In [31]:
def get_node_transform(node, time = 0.0):
    result = get_global_transform(node, time)
    if (node.GetNodeAttribute()):
        gt = get_geometry_transform(node)
        result = result * gt
    return result

In [32]:
print get_node_transform(root_node, 0.0)

In [33]:
def get_node_local_transform(node, time = 0.0):
    transform = get_node_transform(node, time)
    parent = node.GetParent()
    if parent:
        parent_transform = get_node_transform(parent, time)
        inverse_parent_transform = parent_transform.Inverse()
        transform = transform * inverse_parent_transform
    return transform

In [34]:
print get_node_local_transform(root_node, 0.0)

In [35]:
root_node.GetChildCount()

2

In [36]:
child_node = root_node.GetChild(1)

In [37]:
print get_node_local_transform(child_node, 0.0)

In [38]:
def get_node_animated_transform(node, time = 0.0):
    global_transform = node.EvaluateGlobalTransform(FbxTime(time))
    geometry_transform = get_geometry_transform(node)
    return global_transform * geometry_transform
                                                   

In [39]:
print get_node_animated_transform(root_node, 0.5)

In [40]:
root_node??

In [41]:
scene = root_node.GetScene()

In [42]:
scene??

In [43]:
print scene.FindMember.__doc__

FbxCollection.FindMember(FbxClassId, str) -> FbxObject


In [44]:
print scene.FillAnimStackNameArray.__doc__

FbxDocument.FillAnimStackNameArray(FbxStringArray)


In [45]:
string_array = fbx.FbxStringArray()

In [46]:
classid = FbxAnimStack.ClassId

In [47]:
print classid

In [48]:
print scene.GetAnimationEvaluator()

In [49]:
idle_anim_stack = scene.FindMember(FbxAnimStack.ClassId, "Idle")

In [50]:
print idle_anim_stack.GetReferenceTimeSpan().GetStart().GetSecondDouble()

0.0


In [51]:
def get_anim_stacks(scene):
    anim_stack_classid = FbxAnimStack.ClassId
    stack_names = fbx.FbxStringArray()
    scene.FillAnimStackNameArray(stack_names)
    num_stacks = stack_names.GetCount()
    for stack_index in range(0,num_stacks):
        anim_name = stack_names[stack_index]
        if anim_name == None:
            continue
        anim_stack = scene.FindMember(anim_stack_classid, anim_name.Buffer())
        reference_time_span = anim_stack.GetReferenceTimeSpan()
        local_time_span = anim_stack.GetLocalTimeSpan()
        print "Anim Stack ", anim_name
        print "Reference Start, Stop ", anim_stack.GetReferenceTimeSpan().GetStart().GetSecondDouble(), anim_stack.GetReferenceTimeSpan().GetStop().GetSecondDouble()
        print "Local Start, Stop ", anim_stack.GetLocalTimeSpan().GetStart().GetSecondDouble(), anim_stack.GetLocalTimeSpan().GetStop().GetSecondDouble()
        
        

In [52]:
get_anim_stacks(scene)

Anim Stack  Idle
Reference Start, Stop  0.0 1.50000007823
Local Start, Stop  0.0 1.50000007823
Anim Stack  Walk
Reference Start, Stop  0.0 1.80000009386
Local Start, Stop  0.0 1.80000009386
Anim Stack  Jump
Reference Start, Stop  0.0 1.80000009386
Local Start, Stop  0.0 1.80000009386
Anim Stack  Crouch
Reference Start, Stop  0.0 1.80000009386
Local Start, Stop  0.0 1.80000009386


In [53]:
print root_node.GetNodeAttribute()


None


In [54]:
print FbxLight.ClassId

In [55]:
print root_node.GetChild(0).GetNodeAttribute().ClassId

In [92]:
class LayerElement():
    def __init__(self, mesh, layer_index, layer_kind):
        self.mesh = mesh
        self.layer = mesh.GetLayer(layer_index)
        self.elements = self.layer.GetLayerElementOfType(layer_kind)
    def valid(self):
        return (self.layer) and (self.elements)
    def indexed(self):
        return (self.elements.GetReferenceMode() != FbxLayerElement.eDirect)
    def get_element(self, polygonIndex, vertexIndex):
        assert(self.mesh.GetPolygonCount() > polygonIndex)
        assert(self.mesh.GetPolygonSize() > vertexIndex)
        control_point_index = mesh.GetPolygonVertex(polygonIndex, vertexIndex)
        mapping_mode = self.elements.GetMappingMode()
        reference_mode = self.elements.GetReferenceMode() 
        if (mapping_mode == FbxLayerElement.eByControlPoint):
            if (reference_mode == FbxLayerElement.eDirect):
                return self.elements.GetDirectArray().GetAt(control_point_index)
            elif (reference_mode == FbxLayerElement.eIndexToDirect):
                index_array = elements.GetIndexArray()
                assert(index_array.GetCount() == mesh.GetControlPointsCount())
                assert(control_point_index < index_array.GetCount())
                ix = index_array.GetAt(control_point_index)
                direct_array = self.elements.GetDirectArray()
                assert(ix < direct_array.GetCount())
                return self.elements(ix)
        elif (mapping_mode == FbxLayerElement.eByPolygonVertex):
            vertexId = polygonIndex * 3 + vertexIndex;
            if (reference_mode == FbxLayerElement.eDirect):
                direct_array = self.elements.GetDirectArray()
                assert(vertexId < direct_array.GetCount())
                return direct_array.GetAt(vertexId)      
            elif (reference_mode == FbxLayerElement.eIndexToDirect):
                index_array = self.elements.GetIndexArray()
                assert(vertexId < index_array.GetCount())
                ix = index_array.GetAt(vertexId)
                return self.elements.GetDirectArray().GetAt(ix)
        elif (mapping_mode == FbxLayerElement.eByPolygon):
            if (reference_mode == FbxLayerElement.eDirect):
                direct_array = self.elements.GetDirectArray()
                assert(polygonIndex < direct_array.GetCount())
                return direct_array.GetAt(polygonIndex)
            elif (reference_mode == FbxLayerElement.eIndexToDirect):
                index_array = self.elements.GetIndexArray()
                assert(polygonIndex < index_array.GetCount())
                ix = index_array.GetAt(polygonIndex)
                return self.elements.GetDirectArray().GetAt(ix)
        elif (mapping_mode == FbxLayerElement.eAllSame):
            if (reference_mode == FbxLayerElement.eDirect):
                direct_array = self.elements.GetDirectArray()
                return direct_array.GetAt(0)
            elif (reference_mode == FbxLayerElement.eIndexToDirect):
                index_array = self.elements.GetIndexArray()
                ix = index_array.GetAt(0)
                return self.elements.GetDirectArray().GetAt(ix)
        else:
            print "Error - unhandled mapping mode!"
        return None
    def get_vertex_element(self, vertexIndex):
        mapping_mode = self.elements.GetMappingMode()
        reference_mode = self.elements.GetReferenceMode()  
        if (mapping_mode == FbxLayerElement.eByControlPoint):
            if (reference_mode == FbxLayerElement.eDirect):
                return self.elements.GetDirectArray().GetAt(vertexIndex)
            elif (reference_mode == FbxLayerElement.eIndexToDirect):
                ix = self.elements.GetIndexArray().GetAt(vertexIndex)
                return self.elements.GetDirectArray().GetAt(ix)
        elif (mapping_mode == FbxLayerElement.eByPolygonVertex):
            pass
        elif (mapping_mode == FbxLayerElement.eByPolygon):
            pass
        elif (mapping_mode == FbxLayerElement.eByEdge):
            pass
        elif (mapping_mode == FbxLayerElement.eAllSame):
            return self.elements.GetDirectArray().GetAt(vertexIndex)
        return None
    def get_polygon_element(self, polygonIndex):
        mapping_mode = self.elements.GetMappingMode()
        reference_mode = self.elements.GetReferenceMode()  
        if (mapping_mode == FbxLayerElement.eByControlPoint):
            if (reference_mode == FbxLayerElement.eDirect):
                return self.elements.GetDirectArray().GetAt(plygonIndex)
            elif (reference_mode == FbxLayerElement.eIndexToDirect):
                ix = self.elements.GetIndexArray().GetAt(polygonIndex)
                return self.elements.GetDirectArray().GetAt(ix)
        elif (mapping_mode == FbxLayerElement.eByPolygonVertex):
            pass
        elif (mapping_mode == FbxLayerElement.eByPolygon):
            if (reference_mode == FbxLayerElement.eDirect):
                return self.elements.GetDirectArray().GetAt(plygonIndex)
            elif (reference_mode == FbxLayerElement.eIndexToDirect):
                ix = self.elements.GetIndexArray().GetAt(polygonIndex)
                return self.elements.GetDirectArray().GetAt(ix)    
        elif (mapping_mode == FbxLayerElement.eByEdge):
            pass
        elif (mapping_mode == FbxLayerElement.eAllSame):
            if (reference_mode == FbxLayerElement.eDirect):
                direct_array = self.elements.GetDirectArray()
                return direct_array.GetAt(0)
            elif (reference_mode == FbxLayerElement.eIndexToDirect):
                index_array = self.elements.GetIndexArray()
                ix = index_array.GetAt(0)
                return self.elements.GetDirectArray().GetAt(ix)
        return None

  
            

In [90]:
def parse_mapping_mode(mode):
    mapping_types = [ "None", "By Control Point", "By Polygon Vertex", "By Polygon", "By Edge", "All Same" ]
    print mode
    print mapping_types[int(mode)]

In [77]:
def parse_reference_mode(mode):
    reference_types = [ "None", "Direct", "IndexToDirect"]
    print mode
    print reference_types[int(mode)]

In [78]:
def parse_control_points(node, mesh):
    control_points_count = mesh.GetControlPointsCount()
    control_points = mesh.GetControlPoints()
    print "Mesh has %d control points " % (control_points_count)
    layer_count = mesh.GetLayerCount()
    for i in range(layer_count):
        elements = LayerElement(mesh, i, FbxLayerElement.eNormal)
        normals = mesh.GetLayer(i).GetNormals()
        if normals:
            if (normals.GetMappingMode() == FbxLayerElement.eByControlPoint):
                if (normals.GetReferenceMode() == FbxLayerElement.eDirect):
                    print "Layer %d has %d normals " % (i, len(normals.GetDirectArray()))
                else:
                     print "Unexpected Reference Mode"                                           
            else:
                print "Unexpected Mapping Mode"
                print "Layer %d Mode %s Reference %s " % (i, parse_mapping_mode(normals.GetMappingMode()), parse_reference_mode(normals.GetReferenceMode()))
                print "Wut ", normals.GetDirectArray().GetCount()
        else:
            print "Layer %d has no normals" % (i)
                                                   

In [79]:
def parse_layers(node, mesh):
    print "Mesh has %d layers and %d materials " % (mesh.GetLayerCount(), node.GetMaterialCount())
    material_count = node.GetMaterialCount()
    layer_count = mesh.GetLayerCount()
    for l in range(layer_count):
            print "In Layer %d " % (l)
            material = mesh.GetLayer(l).GetMaterials()
            if material:
                if (material.GetReferenceMode() == FbxLayerElement.eDirect):
                    print "Material is directly indexed"
                else:
                    print "Material is indirectly indexed"
                    index_array = material.GetIndexArray()
                    index_array_count = index_array.GetCount()
                    for i in range(index_array_count):
                        print "Index %d : %d " % (i, index_array[i])
            else:
                print "No material"


In [96]:
def parse_materials(node, mesh):
    polygon_count = mesh.GetPolygonCount()
    layer_count = mesh.GetLayerCount()
    material = LayerElement(mesh, 0, FbxLayerElement.eMaterial)
    print dir(material)
    if (not(material.valid())):
        print "No materials"
        return None
    materials_used = []
    for i in range(polygon_count):
        material_connection_index = material.get_polygon_element(i) 
        if (not(material_connection_index in materials_used)):
            materials_used = materials_used + [ material_connection_index ]
    print "Mesh has %d materials" % (len(materials_used))
    uvset_names = []
    for l in range(layer_count):
        print "Layer %d " % (l)
        uvset_count = mesh.GetLayer(l).GetUVSetCount()
        print "Layer has %d uvsets " % (uvset_count)
        if (uvset_count != 0):
            uvsets = mesh.GetLayer(l).GetUVSets()
            for j in range(uvset_count):
                set_name = uvsets[j].GetName()
                print dir(set_name)
                print "Layer %d Set %d Name %s " % (l, j, set_name)
                if (not(set_name in uvset_names)):
                    uvset_names = uvset_names + [ set_name ]
    print materials_used
    print uvset_names
    return (materials_used, uvset_names)
                


In [81]:
def process_mesh(node):
    mesh = node.GetNodeAttribute()
    # print dir(mesh)
    mesh.GenerateNormals()
    mesh.GenerateTangentsDataForAllUVSets()
    print "Mesh has %d layers" % ( mesh.GetLayerCount() )
    polygon_count = mesh.GetPolygonCount()
    parse_materials(node,mesh)
    parse_control_points(node, mesh)
    parse_layers(node, mesh)

In [82]:
def process_bone(node):
    pass

In [83]:
def process_marker(node):
    pass

In [84]:
def classify_node(node):
    kinds = { FbxNodeAttribute.eMarker: "Marker",   FbxNodeAttribute.eSkeleton: "Bone", FbxNodeAttribute.eMesh: "Mesh", FbxNodeAttribute.eNurbs: "Nurbs", FbxNodeAttribute.ePatch: "Patch", FbxNodeAttribute.eCamera: "Camera" }
    result = node.GetNodeAttribute()
    if result:
        node_kind = result.GetAttributeType()
        if kinds.has_key(node_kind):
            print "Is A ", kinds[node_kind]
        else:
            print "Unknown Type ", node_kind
    if (result) and (node_kind == FbxNodeAttribute.eMesh):
        process_mesh(node)
    return result

In [85]:
def walk_nodes_aux(node,fn):
    for i in range(node.GetChildCount()):
        print node.GetChild(i).GetName()
        fn(node.GetChild(i))
        walk_nodes_aux(node.GetChild(i), fn)
        
def walk_nodes(scene, fn):
    root = scene.GetRootNode()
    fn(root)
    walk_nodes_aux(root, fn)

In [97]:
walk_nodes(root_node.GetScene(), classify_node)

Character
Is A  Mesh
Mesh has 2 layers
['__doc__', '__init__', '__module__', 'elements', 'get_element', 'get_polygon_element', 'get_vertex_element', 'indexed', 'layer', 'mesh', 'valid']
Mesh has 1 materials
Layer 0 
Layer has 1 uvsets 
['__add__', '__class__', '__contains__', '__delattr__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__getslice__', '__gt__', '__hash__', '__init__', '__le__', '__len__', '__lt__', '__mod__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmod__', '__rmul__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '_formatter_field_name_split', '_formatter_parser', 'capitalize', 'center', 'count', 'decode', 'encode', 'endswith', 'expandtabs', 'find', 'format', 'index', 'isalnum', 'isalpha', 'isdecimal', 'isdigit', 'islower', 'isnumeric', 'isspace', 'istitle', 'isupper', 'join', 'ljust', 'lower', 'lstrip', 'partition', 'replace', 'rfind', 'rindex', 'rjust', 'rpartitio